# cross

> Fill in a module description here

In [1]:
#| default_exp cross

In [2]:
#| hide
from nbdev.showdoc import *

In [3]:
#| hide
import jax
from jax import vmap
import jax.numpy as jnp
from chewc.sp import SimParam
from chewc.population import quick_haplo, Population
from chewc.trait import add_trait_a, TraitCollection
from chewc.phenotype import set_pheno
from chewc.meiosis import *

# --- 1. JAX Setup ---
key = jax.random.PRNGKey(42)

# --- 2. Define the Genome's "Blueprint" ---
n_chr = 3
n_loci_per_chr = 100
ploidy = 2
gen_map = jnp.array([jnp.linspace(0, 1, n_loci_per_chr) for _ in range(n_chr)])
centromeres = jnp.full(n_chr, 0.5)

# --- 3. Instantiate Initial Simulation Parameters ---
SP = SimParam(
    gen_map=gen_map,
    centromere=centromeres,
    ploidy=ploidy
)

key, pop_key = jax.random.split(key)
n_founders = 50

founder_pop = quick_haplo(
    key=pop_key,
    sim_param=SP,
    n_ind=n_founders,
    inbred=False
)

SP = SP.replace(founderPop=founder_pop)
SP.founderPop.geno.shape # (50, 3, 2, 100) individuals,chromosomes, ploidy, markers


(50, 3, 2, 100)

In [4]:
#| export

# chewc/cross.py

import jax
from jax import vmap
import jax.numpy as jnp
from functools import partial

# Assuming these imports are correctly set up in your project structure
from chewc.sp import SimParam
from chewc.population import Population
from chewc.meiosis import meiosis_for_one_cross

@partial(jax.jit, static_argnames=("n_chr",))
def _make_cross_geno(
    key: jax.random.PRNGKey,
    mothers_geno: jnp.ndarray,
    fathers_geno: jnp.ndarray,
    n_chr: int,
    gen_map: jnp.ndarray,
    recomb_param_v: float
) -> jnp.ndarray:
    """
    (Internal JIT-compiled core) Creates progeny genotypes from parent genotypes.

    This function contains only JAX-traceable operations, making it ideal for
    `jax.jit`. It takes JAX arrays as input and returns a JAX array.

    Args:
        key: A JAX random key. A unique key must be provided for each
             batch of crosses.
        mothers_geno: Genotypes of the mothers. Shape: (nCrosses, nChr, ploidy, nLoci).
        fathers_geno: Genotypes of the fathers. Shape: (nCrosses, nChr, ploidy, nLoci).
        n_chr: The number of chromosomes (static argument).
        gen_map: The genetic map.
        recomb_param_v: The 'v' interference parameter for recombination.

    Returns:
        A JAX array of progeny genotypes. Shape: (nCrosses, nChr, ploidy, nLoci).
    """
    # Vectorize the single-cross meiosis function to run all crosses in parallel.
    # `in_axes` maps over the first dimension (the "cross" dimension) of the
    # keys and parent genotypes, while broadcasting the static parameters.
    vmapped_cross_creator = vmap(
        meiosis_for_one_cross,
        in_axes=(0, 0, 0, None, None, None)
    )

    n_crosses = mothers_geno.shape[0]
    cross_keys = jax.random.split(key, n_crosses)

    progeny_geno = vmapped_cross_creator(
        cross_keys,
        mothers_geno,
        fathers_geno,
        n_chr,
        gen_map,
        recomb_param_v
    )
    return progeny_geno


def make_cross(
    key: jax.random.PRNGKey,
    pop: Population,
    cross_plan: jnp.ndarray,
    sp: SimParam,
    next_id_start: int
) -> Population:
    """
    (Public-facing) Creates progeny from a planned series of crosses.

    This function handles the "CPU-side" logic: preparing data from the main
    Population object, calling the JIT-compiled core `_make_cross_geno`, and
    then assembling the results into a new Population object with updated metadata.

    Args:
        key: A single JAX random key.
        pop: The parent population.
        cross_plan: A 2D array of shape (nCrosses, 2) with mother/father iids.
        sp: The simulation parameters.
        next_id_start: The starting integer for the new individuals' public IDs.


    Returns:
        A new Population object for the progeny.
    """
    n_crosses = cross_plan.shape[0]
    key_geno, key_sex = jax.random.split(key)

    # 1. Prepare JAX arrays for the JIT-compiled function
    mother_iids = cross_plan[:, 0]
    father_iids = cross_plan[:, 1]
    mothers_geno = pop.geno[mother_iids]
    fathers_geno = pop.geno[father_iids]

    # 2. Call the highly-optimized, JIT-compiled core function
    progeny_geno = _make_cross_geno(
        key_geno,
        mothers_geno,
        fathers_geno,
        sp.n_chr,
        sp.gen_map,
        sp.recomb_params[0]
    )

    # 3. Handle CPU-side logic: create new metadata and Population object
    new_public_ids = jnp.arange(next_id_start, next_id_start + n_crosses)
    new_iids = jnp.arange(n_crosses) # Internal IDs are always 0-indexed for the new pop
    mother_public_ids = pop.id[mother_iids]
    father_public_ids = pop.id[father_iids]

    progeny_pop = Population(
        geno=progeny_geno,
        id=new_public_ids,
        iid=new_iids,
        mother=mother_public_ids,
        father=father_public_ids,
        sex=jax.random.choice(key_sex, jnp.array([0, 1], dtype=jnp.int8), (n_crosses,)),
        pheno=jnp.zeros((n_crosses, sp.n_traits)), # Initialize with correct shape
        fixEff=jnp.zeros(n_crosses, dtype=jnp.float32),
        bv=jnp.zeros((n_crosses, sp.n_traits)) # Initialize with correct shape
    )

    return progeny_pop

In [5]:
#| hide
import nbdev; nbdev.nbdev_export()